In [3]:
from gru.gru_v3 import EncoderDecoder
from gru.dataset import GRUDataset
from gru import example_printer
from gru.cce import ConsciousCrossEntropy
import torch
import wandb
import pandas as pd
import torch.nn as nn
from vectorizer import SELFIESVectorizer, determine_alphabet
import random
import torch
model = EncoderDecoder

device = 'cuda'
print(device)

# Set hyperparameters
encoding_size = 512
hidden_size = 512
num_layers = 1
learn_rate = 0.0003
dropout = 0 # dropout must be equal 0 if num_layers = 1
teacher_ratio = 0.5

# Init model
model = EncoderDecoder(
    fp_size=4860,
    encoding_size=encoding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    teacher_ratio = teacher_ratio).to(device)

#model.load_state_dict(torch.load('PATH'))
model.load_state_dict(torch.load('./GRU_data/params/v3_w-teacher-w-enumeration-0.pt'))

cuda


<All keys matched successfully>

In [4]:
alphabet = pd.read_csv('./GRU_data/alphabet.txt', header=None).values.flatten()

In [15]:
from split import scaffold_split

df = pd.read_parquet('./GRU_data/combned_dataset.parquet')
vectorizer = SELFIESVectorizer(alphabet, pad_to_len=128)

train_size = 0.9
_, test_df = scaffold_split(df, train_size)

test_df.to_parquet('./GRU_data/test_dataset.parquet')
test_dataset = GRUDataset(test_df, vectorizer)

print("Dataset size:", len(dataset))
print("Test size:", len(test_dataset))

grouping compounds by murcko scaffold...


100%|███████████████████████████████| 1222179/1222179 [07:27<00:00, 2729.65it/s]


sorting keys...
splitting...


100%|██████████████████████████████| 393587/393587 [00:00<00:00, 2307488.65it/s]


Dataset size: 1222179
Test size: 121524


In [4]:
from torch.utils.data import DataLoader
batch_size = 256
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, drop_last=True)

In [5]:
x, y = next(iter(train_loader))
x = x.to(device)
y = y.to(device)

In [185]:
out = model(x, y, teacher_forcing=False)
softmax = nn.Softmax(dim=2)
out = softmax(out)
out = out.detach().cpu().numpy()
target = y.detach().cpu().numpy()

out.shape

(256, 128, 42)

In [172]:
import selfies as sf
import rdkit.Chem as Chem

preds = []
targets = []

for n in range(256):
    selfie_out = vectorizer.devectorize(out[n], remove_special=True)
    selfie_target = vectorizer.devectorize(target[n], remove_special=True)
    smiles_out = sf.decoder(selfie_out)
    smiles_target = sf.decoder(selfie_target)
    mol_out = Chem.MolFromSmiles(smiles_out)
    mol_target = Chem.MolFromSmiles(smiles_target)
    preds.append(mol_out)
    targets.append(mol_target)

In [283]:
idx += 1
print(idx)
Chem.Draw.MolsToImage([targets[idx], preds[idx]], subImgSize=(400, 400), legends=None)

256


IndexError: list index out of range